In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow.examples.tutorials.mnist import input_data

# 그래프를 초기화해보자! 
tf.reset_default_graph() 

# Data Loading
mnist = input_data.read_data_sets("./data/mnist",one_hot=True)

# placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
keep_rate = tf.placeholder(dtype=tf.float32) # drop out 비율

# Convolution Layer
x_img = tf.reshape(X,[-1,28,28,1])  # -1: 몇장인지 모름    # convolution하려면 무조건 4차원이어야 함!
W1 = tf.Variable(tf.random_normal(shape=[3,3,1,32]))  #  32개의 filter map
    # 여기서는 자비에르초기법 하는거 아님. W의 초기값을 랜덤으로 받는다는 건데 그건 머신러닝때. 지금은 filter를 랜덤으로 잡는 과정이므로 랜덤해야한다.
L1 = tf.nn.conv2d(x_img, W1, strides=[1,1,1,1], padding="SAME")
L1 = tf.nn.relu(L1)  # convolution 하고 relu 해줘야 함(값 너무 커지지 않게)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
# -> 14*14 크기가 32개 나옴

W2 = tf.Variable(tf.random_normal(shape=[3,3,32,64]))  # 32: 앞에서 나오는 channel 수
    # 3*3 크기의 32 depth를 64번 filter하겠다
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding="SAME")
L2 = tf.nn.relu(L2)  
L2 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")    
# -> 7*7 크기가 64개 나옴  


# 이렇게 만든 데이터를 FC Layer에 넣어서 학습해야 한다.
L2 = tf.reshape(L2,[-1,7*7*64])

W3 = tf.get_variable("weight3", shape=[7*7*64,256],
                    initializer=tf.contrib.layers.xavier_initializer())
############################################################################

b3 = tf.Variable(tf.random_normal([256]),name="bias3")
_layer3 = tf.nn.relu(tf.matmul(L2,W3)+b3)
layer3 = tf.nn.dropout(_layer3, keep_prob=keep_rate) # 256개  output을 다 뽑아내지 않겠다. node를 아예 삭제하는게 아니라 기능을 상실시키는 것
                                         # rate=0 하면 다 살아있는거   0.3하면 30% 죽이는거 -> 다음으로 30프로 죽여서 넘기겠다.
     # drop_rate가 cpu_env에서는 keep_rate
W4 = tf.get_variable("weight4", 
                     shape=[256,256], 
                     initializer = tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([256]),name="bias4")
_layer4 = tf.nn.relu(tf.matmul(layer3,W4)+b4)
layer4 = tf.nn.dropout(_layer4,keep_prob=keep_rate) 

W5 = tf.get_variable("weight5", 
                     shape=[256,10], 
                     initializer = tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]),name="bias5")  # 맨 마지막에는 dropout해주지 않는다. -> 다 넘겨야 하므로
logit = tf.matmul(layer4,W5)+b5
H = tf.nn.relu(logit)  

# cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,labels=Y))  

# train
#train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)  # 이거도 자주쓰임. 크게 성능의 향상 보기엔 어렵지만 좀더 성능 좋다는게 일반적인 생각

# session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
num_of_epoch =30  # 반복횟수
batch_size = 100

for step in range(num_of_epoch): 
    num_of_iter = int(mnist.train.num_examples/batch_size)
    cost_val = 0
    
    for i in range(num_of_iter):
        batch_x, batch_y = mnist.train.next_batch(batch_size)  # x,y 100개씩 떼어오기
        _, cost_val = sess.run([train,cost],feed_dict = {X:batch_x,
                                                         Y:batch_y,
                                                         keep_rate:0.7}) # 30% 끄고 학습해라
    if step%3==0:
        print("cost:{}".format(cost_val))

C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\ten

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz


InvalidArgumentError: logits and labels must be same size: logits_size=[50,10] labels_size=[100,10]
	 [[Node: softmax_cross_entropy_with_logits = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](softmax_cross_entropy_with_logits/Reshape, softmax_cross_entropy_with_logits/Reshape_1)]]

Caused by op 'softmax_cross_entropy_with_logits', defined at:
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\ipykernel\kernel